In this notebook, we will expand the current calcium data pipeline by creating a computed table for analysis.

We would like to compute the correlation between the pupil size and locomotion speed of the animal

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import datajoint as dj
import calcium as ca

In [ ]:
schema = dj.schema('YOUR_USERNAME_analysis')

In [ ]:
dj.Di(ca)

Get all information by joining tables...

In [ ]:
combined = ca.Pupil * ca.Treadmill * ca.ScanInfo

Plot for one entry for visualization

In [ ]:
key = combined.fetch('KEY')[0]
r, v, f =(combined & key).fetch1('pupil_r', 'treadmill_vel', 'fps', squeeze=True)
t = np.r_[:r.size]/f

plt.plot(t, r)
plt.plot(t, abs(v)*10)

plt.legend(('pupil radius', 'running speed'));

Now let's create a computed table to compute and store the pupil correlation of all data.

In [ ]:
@schema
class PupilCorr(dj.Computed):
    definition = """
    -> ca.Pupil
    -> ca.Treadmill 
    -> ca.ScanInfo
    ---
    pupil_corr :  float   # correlation between pupil radius and locomotion velccity
    """
    
    def make(self, key):
        # define your computation for one entry of the current table.
        r, v, f =(ca.Pupil * ca.Treadmill * ca.ScanInfo & key).fetch1(
            'pupil_r', 'treadmill_vel', 'fps', squeeze=True)
        t = np.r_[:r.size]/f
        v = abs(v)
        v = v- np.nanmean(v, keepdims=True)
        r = r- np.nanmean(r, keepdims=True)
        r = r / np.sqrt(np.nanmean(r**2))
        v = v / np.sqrt(np.nanmean(v**2))
        c = np.nanmean(r * v)
        self.insert1(dict(key, pupil_corr=c))
        

In [ ]:
PupilCorr.key_source

In [ ]:
PupilCorr.populate(display_progress=True, suppress_errors=False,
                   reserve_jobs=True)

In [ ]:
PupilCorr()

**What does `populate` do?** 

It does two major things:  
1. From the table definition, get the keys that needs to computed, which we called `key_source`. By default, it would be the join result of the primary dependent tables minus the once has been computed.  
2. Call `make` function defined in the class, and compute one by one, with each individual key from the `key_source`